In [ ]:
%load_ext autoreload
%autoreload 2

# Exercise 2

<img src="./images/02.png" width=800>

In [2]:
import requests, zipfile, io
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import mlflow
import torch.nn as nn
import torch.optim as optim
from utils import (train_network, roc_auc_score_micro_wrapper, 
                accuracy_score_wrapper,f1_score_wrapper,
                weight_reset, set_seed)
from torchinfo import summary

/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_04/utils.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
os.environ['MLFLOW_TRACKING_URI'] = './mlruns04_2'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [4]:
mlflow.set_experiment('Exercise_2')

<Experiment: artifact_location='/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_04/mlruns_2/830193463711506022', creation_time=1749121088476, experiment_id='830193463711506022', last_update_time=1749121088476, lifecycle_stage='active', name='Exercise_2', tags={}>

In [5]:
zip_file_url =  "https://download.pytorch.org/tutorial/data.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
# z = zipfile.ZipFile('./data.zip')
# z.extractall()

## Datasets and DataLoaders

In [6]:
namge_language_data = {}

#We will use some code to remove UNICODE tokens to make life easy for us processing wise
#e.g., convert something like "Ślusàrski" to Slusarski
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)
alphabet = {}
for i in range(n_letters):
    alphabet[all_letters[i]] = i
    
# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [7]:
class LanguageNameDataset_inferred_unicode(Dataset):
    def __init__(self, zipfile, vocabulary=None, unicode=False):
        self.namge_language_data = {}
        self.unicode_or_not(z=zipfile, unicode=unicode)
        self.label_names = [x for x in self.namge_language_data.keys()]
        self.data = []
        self.labels = []
        self.vocabulary = vocabulary
        for y, language in enumerate(self.label_names):
            for sample in self.namge_language_data[language]:
                self.data.append(sample)
                self.labels.append(y)
        if vocabulary is None:
            vocabulary_set = {char
                for names in self.data
                for char in names}
            vocabulary = {y:x
            for x, y in enumerate(vocabulary_set)
            }
        self.vocabulary = vocabulary
    def __len__(self):
        return len(self.data)
    
    def string2inputvector(self, input_string):
        T = len(input_string)
        name_vec = torch.zeros((T), dtype=torch.long)
        for pos, character in enumerate(input_string):
            name_vec[pos] = self.vocabulary[character]
        return name_vec
    
    def unicode_or_not(self, z, unicode=False):
        for zip_path in z.namelist():
            if "data/names/" in zip_path and zip_path.endswith(".txt"):
                lang = zip_path[len("data/names/"):-len(".txt")]
                with z.open(zip_path) as myfile:
                    if unicode:
                        lang_names = [line.lower() for line in str(myfile.read(), encoding='utf-8').strip().split("\n")]
                    else:
                        lang_names = [unicodeToAscii(line).lower() for line in str(myfile.read(), encoding='utf-8').strip().split("\n")]
                    self.namge_language_data[lang] = lang_names
                # print(lang, ": ", len(lang_names)) #Print out the name of each language too. 
    
    def __getitem__(self, index):
        name = self.data[index]
        label = self.labels[index]
        
        label_tensor = torch.tensor(label, dtype=torch.long)
        return self.string2inputvector(name), label_tensor

In [8]:
dataset = LanguageNameDataset_inferred_unicode(zipfile=z, vocabulary=alphabet)
dataset_inferred = LanguageNameDataset_inferred_unicode(zipfile=z)
dataset_inferred_unicode = LanguageNameDataset_inferred_unicode(zipfile=z, unicode=True)

print(len(dataset.vocabulary))
print(len(dataset_inferred.vocabulary))
print(len(dataset_inferred_unicode.vocabulary))

57
29
58


In [9]:
def pad_and_pack(batch):
    input_tensors = []
    labels = []
    lengths = []
    for x, y in batch:
        input_tensors.append(x)
        labels.append(y)
        lengths.append(x.shape[0])
    x_padded = torch.nn.utils.rnn.pad_sequence(input_tensors, batch_first=False)
    x_packed = torch.nn.utils.rnn.pack_padded_sequence(x_padded, lengths, batch_first=False, enforce_sorted=False)
    y_batched = torch.as_tensor(labels, dtype=torch.long)
    return x_packed, y_batched

In [10]:
class EmbeddingPackable(nn.Module):
    def __init__(self, embed_layer):
        super().__init__()
        self.embed_layer = embed_layer
    
    def forward(self, input):
        if type(input)== torch.nn.utils.rnn.PackedSequence:
            sequences, lengths = torch.nn.utils.rnn.pad_packed_sequence(
                input.cpu(),
                batch_first=True
            )
            sequences = self.embed_layer(sequences.to(input.data.device))
            return torch.nn.utils.rnn.pack_padded_sequence(
                sequences, lengths.cpu(),
                batch_first=True, enforce_sorted=False
            )
        else:
            return self.embed_layer(input)

In [11]:
batch_size = 256

In [12]:
train_data, test_data = random_split(
    dataset, 
    (len(dataset)-300, 300),
    )
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=pad_and_pack)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=pad_and_pack)

In [13]:
train_data_inferred, test_data_inferred = random_split(
    dataset_inferred, 
    (len(dataset_inferred)-300, 300),
    )
train_loader_inferred = DataLoader(train_data_inferred, batch_size=batch_size, shuffle=True, collate_fn=pad_and_pack)
test_loader_inferred = DataLoader(test_data_inferred, batch_size=batch_size, shuffle=False, collate_fn=pad_and_pack)

In [14]:
train_data_inferred_unicode, test_data_inferred_unicode = random_split(
    dataset_inferred_unicode, 
    (len(dataset_inferred_unicode)-300, 300),
    )
train_loader_inferred_unicode = DataLoader(train_data_inferred_unicode, batch_size=batch_size, shuffle=True, collate_fn=pad_and_pack)
test_loader_inferred_unicode = DataLoader(test_data_inferred_unicode, batch_size=batch_size, shuffle=False, collate_fn=pad_and_pack)

## Model

In [ ]:
class LasttimeStep(nn.Module):
    def __init__(self, rnn_layer=1, bidirectional=False):
        super().__init__()
        self.rnn_layer = rnn_layer
        if bidirectional:
            self.num_bidirectional = 2
        else:
            self.num_bidirectional = 1
    def forward(self, input):
        rnn_output = input[0]
        last_step = input[1]
        if isinstance(last_step, tuple):
            last_step = last_step[0]
        batch_size = last_step.shape[1]
        last_step = last_step.view(self.rnn_layer, self.num_bidirectional, batch_size, -1)
        last_step = last_step[-1]
        return last_step.reshape(batch_size, -1)

In [16]:
D = 64
vocab_size = len(dataset.vocabulary)
hidden_nodes = 256
classes = len(dataset.label_names)

rnn = nn.Sequential(
    EmbeddingPackable(nn.Embedding(vocab_size, D)),
    nn.RNN(D, hidden_size=hidden_nodes, batch_first=True),
    LasttimeStep(),
    nn.Linear(hidden_nodes, classes),
)

In [17]:
D = 64
vocab_size = len(dataset_inferred.vocabulary)
hidden_nodes = 256
classes = len(dataset_inferred.label_names)

rnn_inferred = nn.Sequential(
    EmbeddingPackable(nn.Embedding(vocab_size, D)),
    nn.RNN(D, hidden_size=hidden_nodes, batch_first=True),
    LasttimeStep(),
    nn.Linear(hidden_nodes, classes),
)

In [18]:
D = 64
vocab_size = len(dataset_inferred_unicode.vocabulary)
hidden_nodes = 256
classes = len(dataset_inferred_unicode.label_names)

rnn_inferred_unicode = nn.Sequential(
    EmbeddingPackable(nn.Embedding(vocab_size, D)),
    nn.RNN(D, hidden_size=hidden_nodes, batch_first=True),
    LasttimeStep(),
    nn.Linear(hidden_nodes, classes),
)

## Training

In [19]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
epochs = 20
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size
    }

In [20]:
datasets = (dataset, dataset_inferred, dataset_inferred_unicode)
train_loaders = (train_loader ,train_loader_inferred, train_loader_inferred_unicode)
test_loaders = (test_loader ,test_loader_inferred, test_loader_inferred_unicode)
models = (rnn, rnn_inferred, rnn_inferred_unicode)
name_experiment = ('assumed_vocabulary', 'inferred_vocabulary', 'inferred_unicode_vocabulary')

In [21]:
for i in range(0, 3):
    print(i)
    model = models[i]
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    params['optimizer'] = optimizer.defaults
    
    train_loader = train_loaders[i]
    test_loader = test_loaders[i]
    params['vocabulary'] = datasets[i].vocabulary
    with open('model_summary.txt', 'w') as f:
        f.write(str(summary(model)))
    with mlflow.start_run(nested=True, run_name=name_experiment[i]):
        mlflow.log_artifact('model_summary.txt')
        mlflow.log_params(params)
    
        batch_one_train = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=train_loader,
            valid_loader=test_loader,
            epochs=epochs,
            score_funcs=score_funcs,
            device=device,
        )

0


Epoch: 100%|██████████| 20/20 [01:33<00:00,  4.67s/it]


1


Epoch: 100%|██████████| 20/20 [01:04<00:00,  3.23s/it]


2


Epoch: 100%|██████████| 20/20 [01:14<00:00,  3.75s/it]


<img src="./images/E2_train_acc.png">

<img src="./images/E2_train_loss.png">

<img src="./images/E2_valid_acc.png">

<img src="./images/E2_valid_loss.png">

<img src="./images/E2_time_epoch.png">

<img src="./images/E2_valid_acc_time.png">